## Crop and Fertilizer Recommendation System

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Loading Datasset
crop_data = pd.read_csv("dataset/Crop_recommendation.csv")
fertilizer_data = pd.read_csv("dataset/Fertilizer Prediction.csv")

In [3]:
# Encode categorical labels for crops
crop_encoder = LabelEncoder()
crop_data['label'] = crop_encoder.fit_transform(crop_data['label'])

In [4]:
# Prepare Crop Recommendation Model
y_crop = crop_data['label']
X_crop = crop_data.drop(columns=['label'])
X_train, X_test, y_train, y_test = train_test_split(X_crop, y_crop, test_size=0.2, random_state=42)
crop_model = RandomForestClassifier(n_estimators=100, random_state=42)
crop_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [5]:
#  Fertilizer Data
fertilizer_data.rename(columns={"Temparature": "Temperature", "Phosphorous": "Phosphorus"}, inplace=True)
soil_encoder = LabelEncoder()
crop_type_encoder = LabelEncoder()
fertilizer_name_encoder = LabelEncoder()

In [6]:
fertilizer_data['Soil Type'] = soil_encoder.fit_transform(fertilizer_data['Soil Type'])
fertilizer_data['Crop Type'] = crop_type_encoder.fit_transform(fertilizer_data['Crop Type'])
fertilizer_data['Fertilizer Name'] = fertilizer_name_encoder.fit_transform(fertilizer_data['Fertilizer Name'])

In [7]:

# Prepare Fertilizer Recommendation Model
y_fertilizer = fertilizer_data['Fertilizer Name']
X_fertilizer = fertilizer_data.drop(columns=['Fertilizer Name'])
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_fertilizer, y_fertilizer, test_size=0.2, random_state=42)
fertilizer_model = RandomForestClassifier(n_estimators=100, random_state=42)
fertilizer_model.fit(X_train_f, y_train_f)

RandomForestClassifier(random_state=42)

In [8]:
with open("crop_model.pkl", "wb") as f:
    pickle.dump(crop_model, f)
with open("fertilizer_model.pkl", "wb") as f:
    pickle.dump(fertilizer_model, f)

In [13]:
def recommend_crop_fertilizer(N, P, K, temperature, humidity, ph, rainfall, soil_type):
    # Predict crop
    crop_prediction = crop_model.predict([[N, P, K, temperature, humidity, ph, rainfall]])
    predicted_crop = crop_encoder.inverse_transform(crop_prediction)[0]

    # Handle missing crops in fertilizer dataset
    available_crops = crop_type_encoder.classes_
    
    if predicted_crop not in available_crops:
        print(f"--------- MISSING: Predicted crop '{predicted_crop}' is not in the fertilizer dataset ----------.")
        # Find the closest available crop (optional)
        closest_crop = min(available_crops, key=lambda x: len(set(x) & set(predicted_crop)))
        print(f"Suggesting closest available crop : {closest_crop}")
        predicted_crop = closest_crop  # Use the closest match

    # Transform soil and crop type safely
    try:
        soil_encoded = soil_encoder.transform([soil_type])[0]
        crop_encoded = crop_type_encoder.transform([predicted_crop])[0]
    except ValueError as e:
        print(f"Error: {e}")
        return None, None

    # Predict fertilizer
    fertilizer_prediction = fertilizer_model.predict([[temperature, humidity, 50, soil_encoded, crop_encoded, N, K, P]])
    recommended_fertilizer = fertilizer_name_encoder.inverse_transform(fertilizer_prediction)[0]

    return predicted_crop, recommended_fertilizer

In [14]:
# Example Usage
result = recommend_crop_fertilizer(90, 42, 43, 20.8, 82.0, 6.5, 202.9, 'Sandy')

if result[0] is not None and result[1] is not None:
    print(f"Recommended Crop: {result[0]} &, Recommended Fertilizer: {result[1]}")
else:
    print("No recommendation available due to missing data.")


--------- MISSING: Predicted crop 'rice' is not in the fertilizer dataset ----------.
Suggesting closest available crop : Cotton
Recommended Crop: Cotton &, Recommended Fertilizer: Urea


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
result1 = recommend_crop_fertilizer(80, 35, 40, 25.5, 70.0, 6.8, 200.0, 'Sandy')
print(f"Test 1 → Crop: {result1[0]}, Fertilizer: {result1[1]}")

--------- MISSING: Predicted crop 'coffee' is not in the fertilizer dataset ----------.
Suggesting closest available crop : Paddy
Test 1 → Crop: Paddy, Fertilizer: Urea


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [16]:
result2 = recommend_crop_fertilizer(50, 20, 30, 22.0, 85.0, 6.0, 250.0, 'Clayey')
print(f"Test 2 → Crop: {result2[0]}, Fertilizer: {result2[1]}")

--------- MISSING: Predicted crop 'pomegranate' is not in the fertilizer dataset ----------.
Suggesting closest available crop : Oil seeds
Test 2 → Crop: Oil seeds, Fertilizer: Urea


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [17]:
result3 = recommend_crop_fertilizer(60, 25, 35, 18.0, 75.0, 7.2, 180.0, 'Loamy')
print(f"Test 3 → Crop: {result3[0]}, Fertilizer: {result3[1]}")

--------- MISSING: Predicted crop 'jute' is not in the fertilizer dataset ----------.
Suggesting closest available crop : Paddy
Test 3 → Crop: Paddy, Fertilizer: Urea


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [18]:
result4 = recommend_crop_fertilizer(100, 45, 50, 30.0, 90.0, 5.5, 220.0, 'Black')
print(f"Test 4 → Crop: {result4[0]}, Fertilizer: {result4[1]}")


--------- MISSING: Predicted crop 'papaya' is not in the fertilizer dataset ----------.
Suggesting closest available crop : Cotton
Test 4 → Crop: Cotton, Fertilizer: Urea


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
